In [1]:
import torch
import intel_extension_for_pytorch as ipex
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime


# Custom Libraries
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.helper import to_python_native, gen_experiment_name, set_seed, save_model_state
from utils.plotter import alignment_progress_over_iterations, plot_initial_final_accuracies
from utils.pennylane.model import Qkernel

[W724 17:09:09.718631997 OperatorEntry.cpp:154] Warning: Warning only once for all operators,  other operators may also be overridden.
  Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::_addmm_activation(Tensor self, Tensor mat1, Tensor mat2, *, Scalar beta=1, Scalar alpha=1, bool use_gelu=False) -> Tensor
    registered at /pytorch/build/aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: AutocastCPU
  previous kernel: registered at /pytorch/aten/src/ATen/autocast_mode.cpp:327
       new kernel: registered at /opt/workspace/ipex-cpu-dev/csrc/cpu/autocast/autocast_mode.cpp:112 (function operator())
2025-07-24 17:09:11,390	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-24 17:09:11,576	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook outp

In [2]:
# === Backend Configuration ===
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")

set_seed(42)

device

device(type='cpu')

In [3]:
data_generator = DataGenerator(
        dataset_name= 'checkerboard',
        file_path= 'data/checkerboard_dataset.npy',
    )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset()
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

In [4]:
print(training_data.size())
print(testing_data.size())

torch.Size([30, 2])
torch.Size([30, 2])


In [5]:
kernel = Qkernel(
        device='lightning.qubit',
        n_qubits=4,
        trainable=True,
        input_scaling=True,
        data_reuploading=True,
        ansatz='embedding_paper',
        ansatz_layers=5
    )

In [6]:
agent = TrainModel(
        kernel=kernel,
        training_data=training_data,
        training_labels=training_labels,
        testing_data=testing_data,
        testing_labels=testing_labels,
        optimizer='gd',
        lr= 0.1,
        mclr=0.01,
        cclr=0.01,
        epochs=1000,
        train_method='ccka',
        target_accuracy=0.95,
        get_alignment_every=1500,
        validate_every_epoch=1,
        base_path='./',
        lambda_kao=0.001,
        lambda_co=0.001,
        clusters=8
    )

Epochs:  100


In [7]:
before_metrics = agent.evaluate_test(testing_data, testing_labels, 'before')
before_metrics

proto_labels: [-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1]
len(subcentroids): 16
{-1: [Parameter containing:
tensor([-0.7532,  0.9325], requires_grad=True), Parameter containing:
tensor([-1.0201, -2.7882], requires_grad=True), Parameter containing:
tensor([ 2.9360, -2.5087], requires_grad=True), Parameter containing:
tensor([1.0236, 2.9054], requires_grad=True), Parameter containing:
tensor([-2.9927,  2.9968], requires_grad=True), Parameter containing:
tensor([ 1.0254, -0.9576], requires_grad=True), Parameter containing:
tensor([-2.9673, -0.7056], requires_grad=True), Parameter containing:
tensor([2.7942, 1.3157], requires_grad=True)], 1: [Parameter containing:
tensor([-0.8904,  2.7532], requires_grad=True), Parameter containing:
tensor([ 0.6595, -2.5765], requires_grad=True), Parameter containing:
tensor([-2.8906, -2.8270], requires_grad=True), Parameter containing:
tensor([ 2.8698, -1.0826], requires_grad=True), Parameter containing:
tensor([-0.8495, -0.8127], requires_gr

{'alignment': tensor(0.1708, grad_fn=<SqueezeBackward0>),
 'executions': None,
 'training_accuracy': 1.0,
 'testing_accuracy': 0.9,
 'f1_score': 0.8996662958843159,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}

In [8]:
agent.fit_kernel(training_data, training_labels)

Started Training


(Qkernel(),
 [Parameter containing:
  tensor([[ 0.4272,  1.1133, -0.1624,  0.4983],
          [ 0.6128,  1.3252,  1.5477,  1.3025],
          [ 0.8931,  1.2276,  0.4498,  1.2622],
          [ 1.3164,  0.3024,  0.8658,  0.8592],
          [ 0.6158,  0.3992,  0.9372,  0.5487]], requires_grad=True),
  Parameter containing:
  tensor([[ 1.4855,  2.0281, -0.4422,  2.3168],
          [-0.3033,  0.3754, -1.3517,  1.7436],
          [ 2.3319, -1.5767,  2.7651,  0.0176],
          [ 1.6410,  0.3120,  1.5512, -0.5786],
          [ 1.9828,  0.3802, -1.2007,  0.6556]], requires_grad=True),
  Parameter containing:
  tensor([[-0.8259, -0.3861, -0.8868,  1.8092],
          [-2.6817, -0.8758, -0.7075, -1.5365],
          [ 0.3146, -2.6217,  2.4613, -1.7053],
          [ 2.0351, -0.1621, -0.7131,  1.3693],
          [ 0.4009,  2.0781,  0.2812, -0.8112]], requires_grad=True)],
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 2]
     (1): Parameter containing: [torch.float32 of size 

In [9]:
after = agent.evaluate_test(testing_data, testing_labels, 'before')
after

proto_labels: [-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1]
len(subcentroids): 16
{-1: [Parameter containing:
tensor([-0.6588,  0.9516], requires_grad=True), Parameter containing:
tensor([-1.0098, -2.7658], requires_grad=True), Parameter containing:
tensor([ 2.9144, -2.5141], requires_grad=True), Parameter containing:
tensor([1.0649, 2.8934], requires_grad=True), Parameter containing:
tensor([-2.9802,  2.9718], requires_grad=True), Parameter containing:
tensor([ 1.0381, -0.9792], requires_grad=True), Parameter containing:
tensor([-2.9327, -0.7028], requires_grad=True), Parameter containing:
tensor([2.7956, 1.3191], requires_grad=True)], 1: [Parameter containing:
tensor([-0.9338,  2.7264], requires_grad=True), Parameter containing:
tensor([ 0.6510, -2.5848], requires_grad=True), Parameter containing:
tensor([-2.9092, -2.8314], requires_grad=True), Parameter containing:
tensor([ 2.8550, -1.0992], requires_grad=True), Parameter containing:
tensor([-0.8408, -0.8306], requires_gr

{'alignment': tensor(0.2312, grad_fn=<SqueezeBackward0>),
 'executions': 16000,
 'training_accuracy': 1.0,
 'testing_accuracy': 0.9666666666666667,
 'f1_score': 0.9667037449017427,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}